In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
file_name = "df_count_3_part1.csv"   

In [3]:
 
df = pd.read_csv(file_name , parse_dates=["date"])


C:\Users\oussa\AppData\Local\Temp\ipykernel_4912\1571972478.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name , parse_dates=["date"])


In [4]:
df.shape

(11785664, 10)

In [5]:
# Read the country reference file
df_countries = pd.read_csv("CT_COUNTRIES.csv", sep=";", header=None, names=["code", "lang", "name"])

country_dict = df_countries.set_index("code")["name"].to_dict()

In [6]:
# Define columns to ignore when checking duplicates
cols_ignore = ["RECPTCL_FID", "duration_to_next_step", "total_duration", "id"]
cols_check  = [c for c in df.columns if c not in cols_ignore]

# 1. Identify all duplicate rows based on cols_check
dup_mask = df.duplicated(subset=cols_check, keep=False)

# 2. Number of such duplicate rows
num_duplicates = dup_mask.sum()
print(f"Number of duplicate rows (ignoring specified columns): {num_duplicates}")




Number of duplicate rows (ignoring specified columns): 14890


In [7]:
df = df.drop_duplicates(subset=cols_check, keep="first").reset_index(drop=True)

In [8]:
import pandas as pd

ID_COL    = "MAILITM_FID"
EVENT_COL = "EVENT_TYPE_NM"

# 1) pick the first row per parcel (rows are already in correct order)
first_events = (
    df.groupby(ID_COL, sort=False)[EVENT_COL]
      .first()                 # a Series: index = parcel ID, value = first event
)

# 2) count how many parcels have each first event
event_counts = (
    first_events.value_counts()   # Series: index = event, value = count
               .sort_values(ascending=False)
)

# 3) print the result
print("First-event frequencies:")
for event, count in event_counts.items():
    print(f"{event:40} {count:,}")


First-event frequencies:
Insérer envoi dans sac (Srt)             833,948
Réception d'envoi du client (Srt)        223,534
Expédition d'envoi à l'étranger (EDI-reçu) 1,007


In [9]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

RECPTCL_FID                  1408923
établissement_postal         1471842
next_établissement_postal    7281355
duration_to_next_step        3171616
dtype: int64


In [10]:
df = df.drop(columns=["RECPTCL_FID", "duration_to_next_step", "total_duration"])

In [11]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

établissement_postal         1471842
next_établissement_postal    7281355
dtype: int64


In [12]:

# Assuming 'df' is your DataFrame!

# Identify the rows with missing values
missing_rows = df[df.isnull().any(axis=1)]

# Print the total number of rows that have missing values
print("Total rows with missing values:", missing_rows.shape[0])




Total rows with missing values: 7943805


In [13]:
# Analyze the missing patterns across rows:
# Create a binary pattern for each row where 1 indicates a missing value and 0 indicates non-missing.
missing_pattern = missing_rows.isnull().astype(int)
# Convert each pattern to a tuple to use as a key for groupby
missing_pattern_tuples = missing_pattern.apply(tuple, axis=1)
# Group by the missing pattern and count the number of rows for each pattern
pattern_summary = missing_rows.groupby(missing_pattern_tuples).size().sort_values(ascending=False)

print("\nSummary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):")
print(pattern_summary)




Summary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):
(0, 0, 0, 0, 0, 1, 0)    6471963
(0, 0, 0, 1, 0, 1, 0)     809392
(0, 0, 0, 1, 0, 0, 0)     662450
dtype: int64


In [14]:
distinct_values_count = df.nunique()

distinct_values_count = distinct_values_count.sort_values(ascending=True)

print(distinct_values_count)

EVENT_TYPE_NM                     24
EVENT_TYPE_CD                     24
id                                57
next_établissement_postal       3433
établissement_postal            3566
MAILITM_FID                  1058489
date                         9260169
dtype: int64


In [15]:

# Count the occurrences of each protocol
event_type = df['EVENT_TYPE_NM'].value_counts()

print(event_type)


EVENT_TYPE_NM
Recevoir envoi au bureau de livraison (Ent)                    2248484
Expédier envoi à adresse nationale (Ent)                       1970159
Expédier envoi à adresse nationale (Srt)                       1433458
Insérer envoi dans sac (Srt)                                   1089055
Recevoir envoi au bureau d'échange (Ent)                       1080070
Expédition d'envoi à l'étranger (EDI-reçu)                     1061650
Livraison d'envoi (Ent)                                         967145
Vaine tentative de livraison d'envoi (Ent)                      808366
Transmettre envoi à l'agent de livraison (Ent)                  584783
Réception d'envoi du client (Srt)                               333783
Enregistrer détails d'envoi au bureau d'échange (Srt)           153380
Expédier envoi à la douane (Ent)                                 20919
Recevoir envoi au bureau d'échange (Srt)                         12145
Renvoyer envoi de la douane (Ent)                              

In [16]:

# Count the occurrences of each protocol
proto_counts = df['établissement_postal'].value_counts()

print(proto_counts)


établissement_postal
ALGER GARE                   1757197
ALGER COLIS POSTAUX          1258347
SINGAPOUR                     624269
CONSTANTINE COLIS POSTAUX     479672
ANNABA EL MARSA               310988
                              ...   
ÉGYPTE                             1
THILIDJENE                         1
BAB-EL-ASSA                        1
METIRCHOU                          1
SAIDA-UNIVERSITE                   1
Name: count, Length: 3566, dtype: int64


In [17]:
# Count the occurrences of each protocol
proto_counts = df['next_établissement_postal'].value_counts()

print(proto_counts)



next_établissement_postal
ALGÉRIE                      1061650
SECTION PAQUETS CPX ALGER     463752
ALGER GARE                    367359
CONSTANTINE COLIS POSTAUX     169816
ORAN COLIS POSTAUX            164591
                              ...   
SERIANA-ERIS                       1
AOKAS-20-AOUT-1956                 1
AKBOU-GUELDAMENE                   1
OULED HELLAL                       1
EL BAYADH FRERES FERADJI           1
Name: count, Length: 3433, dtype: int64


In [18]:

# Count the occurrences of each protocol
proto_counts = df['id'].value_counts()

print(proto_counts)

s = proto_counts


id
SG    8811627
NL    2496080
DE     275049
MY     106151
PL      18646
SE      14101
SA      11054
AE      11036
AU       5740
NO       5320
LU       3317
PT       2435
PK       2279
IN       1957
QA       1903
EE       1184
CZ       1153
BR        970
US        912
SI        836
RU        835
OM        664
PS        601
TR        583
NZ        552
HK        476
KZ        463
FR        445
HR        349
LV        220
ES        219
CL        182
FI        141
IQ         98
IT         69
PE         69
TN         62
BA         58
BY         44
XZ         43
BE         39
ID         26
CA         26
CN         23
IE         23
RS         20
BQ         16
JP         16
VU         16
UA         13
MA         12
BF          9
ZA          9
UY          8
CV          8
VN          7
AT          6
Name: count, dtype: int64


In [19]:
# Map the Series index using the country_dict
s.index = s.index.map(lambda code: country_dict.get(code, code))  # Keep unmapped codes as-is
print(s)

id
SINGAPOUR                         8811627
PAYS-BAS                          2496080
ALLEMAGNE                          275049
MALAISIE                           106151
POLOGNE                             18646
SUÈDE                               14101
ARABIE SAOUDITE                     11054
ÉMIRATS ARABES UNIS                 11036
AUSTRALIE                            5740
NORVÈGE                              5320
LUXEMBOURG                           3317
PORTUGAL                             2435
PAKISTAN                             2279
INDE                                 1957
QATAR                                1903
ESTONIE                              1184
TCHÈQUE, RÉPUBLIQUE                  1153
BRÉSIL                                970
ÉTATS-UNIS                            912
SLOVÉNIE                              836
RUSSIE, FÉDÉRATION DE                 835
OMAN                                  664
PALESTINIEN OCCUPÉ, TERRITOIRE        601
TURQUIE                        

In [20]:
print("Min date:", df["date"].min())
print("Max date:", df["date"].max())

Min date: 2020-01-05 08:54:58.627
Max date: 2025-05-21 15:26:16.373


In [21]:
df["date"] = df["date"].str.replace(r'\.\d+$', '', regex=True)
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)

C:\Users\oussa\AppData\Local\Temp\ipykernel_4912\579475394.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)


In [22]:
df = df.sort_values(by=["MAILITM_FID", "date"])

# Calculate time to next step
df["duration_to_next_step"] = df.groupby("MAILITM_FID")["date"].shift(-1) - df["date"]

# Calculate total duration per ID
first_date = df.groupby("MAILITM_FID")["date"].transform("first")
last_date = df.groupby("MAILITM_FID")["date"].transform("last")
df["total_duration"] = last_date - first_date




In [23]:
# Preview the updated dataset
df[["MAILITM_FID", "date", "EVENT_TYPE_NM", "duration_to_next_step", "total_duration"]].head(10)

,MAILITM_FID,date,EVENT_TYPE_NM,duration_to_next_step,total_duration
0,CA000340856PK,2023-01-20 11:33:00,Insérer envoi dans sac (Srt),0 days 16:57:00,9 days 01:19:00
1,CA000340856PK,2023-01-21 04:30:00,Expédition d'envoi à l'étranger (EDI-reçu),3 days 03:52:48,9 days 01:19:00
4768770,CA000340856PK,2023-01-24 08:22:48,Recevoir envoi au bureau d'échange (Ent),0 days 00:08:52,9 days 01:19:00
2582371,CA000340856PK,2023-01-24 08:31:40,Expédier envoi à adresse nationale (Ent),0 days 00:09:08,9 days 01:19:00
2582372,CA000340856PK,2023-01-24 08:40:48,Expédier envoi à adresse nationale (Ent),0 days 00:20:26,9 days 01:19:00
2582373,CA000340856PK,2023-01-24 09:01:14,Recevoir envoi au bureau de livraison (Ent),0 days 00:02:37,9 days 01:19:00
2582374,CA000340856PK,2023-01-24 09:03:51,Expédier envoi à adresse nationale (Srt),0 days 00:21:14,9 days 01:19:00
4768771,CA000340856PK,2023-01-24 09:25:05,Expédier envoi à adresse nationale (Srt),0 days 09:18:00,9 days 01:19:00
4768772,CA000340856PK,2023-01-24 18:43:05,Expédier envoi à adresse nationale (Ent),0 days 04:34:05,9 days 01:19:00
2582375,CA000340856PK,2023-01-24 23:17:10,Expédier envoi à adresse nationale (Ent),0 days 08:20:06,9 days 01:19:00


In [26]:
# Filter rows where total_duration is NOT zero
zero_duration_df = df[df["total_duration"] <= pd.Timedelta(days = 1)]

# Display the first few rows as a check
zero_duration_df.shape


(0, 9)

In [38]:

# Count the occurrences of each protocol
countries = zero_duration_df['id'].value_counts()

print(countries)


Series([], Name: count, dtype: int64)


In [39]:
# Map the Series index using the country_dict
countries.index = countries.index.map(lambda code: country_dict.get(code, code))  
print(countries)

Series([], Name: count, dtype: int64)


In [40]:
# Step 1: Count how many times each ID appears
id_counts = zero_duration_df["MAILITM_FID"].value_counts()

# Step 2: Count how many IDs have a specific count
count_of_counts = id_counts.value_counts().sort_index()

print(count_of_counts)


Series([], Name: count, dtype: int64)


In [41]:
ids_with_0 = id_counts[id_counts == 2].index
df_with_0 = df[df["MAILITM_FID"].isin(ids_with_0)]
df_with_0.head()

,MAILITM_FID,EVENT_TYPE_NM,date,établissement_postal,EVENT_TYPE_CD,next_établissement_postal,id,duration_to_next_step,total_duration


In [42]:

# Count the occurrences of each protocol
event_type = df_with_0['EVENT_TYPE_NM'].value_counts()

print(event_type)



Series([], Name: count, dtype: int64)


In [25]:
# Drop rows where total_duration is zero
df = df[df["total_duration"] != pd.Timedelta(0)]

# Check the shape after dropping
print(df.shape)


(8803577, 9)


In [27]:
df.to_csv(file_name, index=False, encoding='utf-8-sig')